In [1]:
import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image

import code
import os
#import tensorflow.python.platform

import numpy as np
import tensorflow.keras as keras
from keras import layers, models, datasets,regularizers, Model, Input
from keras.layers import concatenate, Conv2D, Conv2DTranspose, BatchNormalization, MaxPooling2D, Dropout
from helpers_images import * 


In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [3]:
NUMBER_TRAINING_EXAMPLES = 100
NUMBER_TEST_EXAMPLES = 50
DATA_LOADED = False

In [4]:
training_data_directory = "../data/training/"
test_data_directory = "../data/test_set_images/"
if not(DATA_LOADED):    
    train_data_orig = extract_train_data(NUMBER_TRAINING_EXAMPLES)
    train_labels_orig =  extract_labels(training_data_directory, NUMBER_TRAINING_EXAMPLES,True)
    DATA_LOADED = True

print(train_data_orig.shape)
print(train_labels_orig.shape)

Loading ../data/training/images/satImage_001.png
Loading ../data/training/images/satImage_002.png
Loading ../data/training/images/satImage_003.png
Loading ../data/training/images/satImage_004.png
Loading ../data/training/images/satImage_005.png
Loading ../data/training/images/satImage_006.png
Loading ../data/training/images/satImage_007.png
File ../data/training/images/satImage_008.png does not exist
Loading ../data/training/images/satImage_009.png
Loading ../data/training/images/satImage_010.png
File ../data/training/images/satImage_011.png does not exist
Loading ../data/training/images/satImage_012.png
Loading ../data/training/images/satImage_013.png
Loading ../data/training/images/satImage_014.png
Loading ../data/training/images/satImage_015.png
Loading ../data/training/images/satImage_016.png
Loading ../data/training/images/satImage_017.png
Loading ../data/training/images/satImage_018.png
Loading ../data/training/images/satImage_019.png
Loading ../data/training/images/satImage_020.

In [5]:
#plt.imshow(train_data_orig[187])

In [6]:
IMG_PATCH_SIZE = 400
train_data = train_data_orig
train_labels = train_labels_orig

In [7]:
def encoding_block(input_, nbr_filters = 16, pooling_ = True, dropout = True):
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_normal', padding='same') (input_)
    conv = BatchNormalization()(conv)
    if dropout:
        conv = Dropout(0.1) (conv)
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_normal', padding='same') (conv)
    conv = BatchNormalization()(conv)
    if pooling_:
        pooling = MaxPooling2D((2,2))(conv)
        return conv, pooling
    return conv

In [8]:
def decoding_block(conv, previous_conv,nbr_filters = 16, dropout = True):
    upsample = Conv2DTranspose(nbr_filters, (2, 2),kernel_regularizer=regularizers.l2(0.01), strides=(2, 2), padding='same') (conv)
    upsample = concatenate([upsample, previous_conv])
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_normal', padding='same') (upsample)
    conv = BatchNormalization() (conv)
    if dropout:
        conv = Dropout(0.1) (conv)
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_normal', padding='same') (conv)
    conv = BatchNormalization() (conv)
    return conv

In [9]:
def create_custom_unet(nbr_filters = 16, dropout = True):
    inputs = Input((IMG_PATCH_SIZE, IMG_PATCH_SIZE, 3))
    conv1, pooling1 = encoding_block(inputs, nbr_filters, True, dropout)
    conv2, pooling2 = encoding_block(pooling1, nbr_filters*2, True,dropout)
    conv3, pooling3 = encoding_block(pooling2, nbr_filters*4, True,dropout)
    conv4, pooling4 = encoding_block(pooling3, nbr_filters*8, True,dropout)
    conv5 = encoding_block(pooling4, nbr_filters*16,False, dropout)

    conv6 = decoding_block(conv5, conv4, nbr_filters*8, dropout)
    conv7 = decoding_block(conv6, conv3, nbr_filters*4, dropout)
    conv8 = decoding_block(conv7, conv2, nbr_filters*2, dropout)
    conv9 = decoding_block(conv8, conv1, nbr_filters, dropout)
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [10]:
from keras import backend as K

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 10,
                          verbose = 1,
                          mode = "min",
                          restore_best_weights = True)
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               mode="min",
                               min_delta=1e-4)

In [12]:
model = create_custom_unet(16, True)
model.summary()
model.compile(optimizer='adam',
            loss=soft_dice_loss,
          metrics=[dice_coef])       

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 400, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 400, 400, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 400, 400, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [13]:
epochs = 200
history = model.fit(train_data, train_labels, epochs=epochs, validation_split = 0.05, batch_size = 8,callbacks = [ earlystopper, lr_reducer])

Epoch 1/200
110/110 [==============================] - 19s 94ms/step - loss: 2.0868 - dice_coef: 0.5367 - val_loss: 1.1453 - val_dice_coef: 0.2901 - lr: 0.0010
Epoch 2/200
110/110 [==============================] - 8s 77ms/step - loss: 0.6555 - dice_coef: 0.6809 - val_loss: 0.6596 - val_dice_coef: 0.6201 - lr: 0.0010
Epoch 3/200
110/110 [==============================] - 9s 78ms/step - loss: 0.5544 - dice_coef: 0.7058 - val_loss: 0.5421 - val_dice_coef: 0.6909 - lr: 0.0010
Epoch 4/200
110/110 [==============================] - 8s 76ms/step - loss: 0.4679 - dice_coef: 0.7461 - val_loss: 0.6505 - val_dice_coef: 0.5443 - lr: 0.0010
Epoch 5/200
110/110 [==============================] - 8s 77ms/step - loss: 0.4368 - dice_coef: 0.7576 - val_loss: 0.4718 - val_dice_coef: 0.7044 - lr: 0.0010
Epoch 6/200
110/110 [==============================] - 9s 78ms/step - loss: 0.4031 - dice_coef: 0.7725 - val_loss: 0.5799 - val_dice_coef: 0.5813 - lr: 0.0010
Epoch 7/200
110/110 [========================

110/110 [==============================] - 9s 85ms/step - loss: 0.1506 - dice_coef: 0.9079 - val_loss: 0.1278 - val_dice_coef: 0.9309 - lr: 1.0000e-05
Epoch 50/200
110/110 [==============================] - 9s 84ms/step - loss: 0.1493 - dice_coef: 0.9091 - val_loss: 0.1270 - val_dice_coef: 0.9316 - lr: 1.0000e-05
Epoch 51/200
110/110 [==============================] - 9s 83ms/step - loss: 0.1496 - dice_coef: 0.9057 - val_loss: 0.1264 - val_dice_coef: 0.9319 - lr: 1.0000e-05
Epoch 52/200
110/110 [==============================] - 9s 83ms/step - loss: 0.1495 - dice_coef: 0.9078 - val_loss: 0.1256 - val_dice_coef: 0.9325 - lr: 1.0000e-05
Epoch 53/200
110/110 [==============================] - 9s 82ms/step - loss: 0.1475 - dice_coef: 0.9105 - val_loss: 0.1256 - val_dice_coef: 0.9323 - lr: 1.0000e-05
Epoch 54/200
110/110 [==============================] - 9s 82ms/step - loss: 0.1474 - dice_coef: 0.9102 - val_loss: 0.1246 - val_dice_coef: 0.9331 - lr: 1.0000e-05
Epoch 55/200
110/110 [=======

110/110 [==============================] - 9s 83ms/step - loss: 0.1339 - dice_coef: 0.9178 - val_loss: 0.1142 - val_dice_coef: 0.9375 - lr: 1.0000e-05
Epoch 100/200
109/110 [============================>.] - ETA: 0s - loss: 0.1350 - dice_coef: 0.9166
Epoch 00100: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
110/110 [==============================] - 9s 82ms/step - loss: 0.1351 - dice_coef: 0.9163 - val_loss: 0.1139 - val_dice_coef: 0.9377 - lr: 1.0000e-05
Epoch 101/200
110/110 [==============================] - 9s 82ms/step - loss: 0.1339 - dice_coef: 0.9178 - val_loss: 0.1124 - val_dice_coef: 0.9392 - lr: 1.0000e-06
Epoch 102/200
110/110 [==============================] - 9s 82ms/step - loss: 0.1325 - dice_coef: 0.9189 - val_loss: 0.1122 - val_dice_coef: 0.9394 - lr: 1.0000e-06
Epoch 103/200
110/110 [==============================] - 9s 82ms/step - loss: 0.1326 - dice_coef: 0.9189 - val_loss: 0.1122 - val_dice_coef: 0.9393 - lr: 1.0000e-06
Epoch 104/200
110/110 

In [14]:
import pathlib
directory_path = pathlib.Path(test_data_directory)

In [15]:
all_images = []
for i in range(1,51):
    image_in_directory = list(directory_path.glob("test_" + str(i) + "/*"))
    all_images.extend(image_in_directory)
img_height = img_width = 608

In [16]:
all_images_arrays = []
for i in range(len(all_images)):
    img = tf.keras.utils.load_img(
        all_images[i], target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    all_images_arrays.append(img_array)
all_images_arrays = numpy.array(all_images_arrays)/255.0

In [17]:
predictions_testing = "./predictions/"
if not os.path.isdir(predictions_testing):
    os.mkdir(predictions_testing)

In [18]:
def predict(k):
    img = all_images_arrays[k]
    extract_1 = img[:400, :400]
    extract_2 = img[:400, -400:]
    extract_3 = img[-400:, :400]
    extract_4 = img[-400:, -400:]
    output_prediction = []
    output_prediction.append(model(np.expand_dims(extract_1 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_2 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_3 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_4 , axis=0)).numpy() )

    output_prediction[0] = output_prediction[0].reshape(400,400)
    output_prediction[1] = output_prediction[1].reshape(400,400)
    output_prediction[2] = output_prediction[2].reshape(400,400)
    output_prediction[3] = output_prediction[3].reshape(400,400)
    gt_img = numpy.zeros([img.shape[0], img.shape[1]])
    gt_img[:400,:400] = output_prediction[0]
    gt_img[:400, -400:] = output_prediction[1]
    gt_img[-400: , :400] = output_prediction[2]
    gt_img[-400: , -400:] = output_prediction[3]
    oimg = make_img_overlay(img, gt_img)
    gt_img8 = img_float_to_uint8(gt_img)          
    gt_img_3c = numpy.zeros((img.shape[0], img.shape[1], 3), dtype=numpy.uint8)
    gt_img_3c[:, :, 0] = gt_img8
    gt_img_3c[:, :, 1] = gt_img8
    gt_img_3c[:, :, 2] = gt_img8
    overlay = make_img_overlay(img, gt_img)
    Image.fromarray(gt_img_3c).save(predictions_testing + "prediction_image_" + str(k+1) + ".png")
    oimg.save(predictions_testing  + "overlay_" + str(k) + ".png")

In [19]:
for i in range(len(all_images_arrays)):
    predict(i)

In [20]:
5

5

In [21]:
model.save("./models/model_unet_0.893_0.941_0.2dropout_0.15validation_0.001l2.h5")